## Check Working Directory

In [1]:
import os

# Display current workspace
current_working_directory = os.getcwd()
print("Aktuelles Arbeitsverzeichnis:", current_working_directory)

Aktuelles Arbeitsverzeichnis: /Users/henrikfliegel/Desktop/Uni/SoSe24/MlClaas/ML-with-CLAAS


## Install Dependencies

In [ ]:
pip install pandas

## Load Data

In [4]:
import pandas as pd

# Laden der Excel-Datei
excel_file = pd.ExcelFile('CLAAS_data/Fact_GJ2017_GJ2022.XLSX')

# Excel-Datei enthält mehrere Tabellenblätter
sheet_names = excel_file.sheet_names  # Liste aller Tabellenblattnamen

print(sheet_names)

# Laden jedes Tabellenblatts in ein eigenes DataFrame
dataframes = {sheet_name: excel_file.parse(sheet_name) for sheet_name in sheet_names}

['Fact_GJ2017', 'Fact_GJ2018', 'Fact_GJ2019', 'Fact_GJ2020', 'Fact_GJ2021', 'Fact_GJ2022']


In [6]:
# Erstellen einer Liste, um alle DataFrames zu sammeln
all_dataframes = []

# Schleife durch alle Tabellenblätter in der Excel-Datei
for sheet_name in excel_file.sheet_names:
    # Laden jedes Tabellenblatts als DataFrame
    df = excel_file.parse(sheet_name)
    # Hinzufügen des DataFrame zur Liste
    all_dataframes.append(df)

# Kombinieren aller DataFrames in der Liste zu einem einzigen DataFrame
combined_df = pd.concat(all_dataframes, ignore_index=True)

combined_df['Monat'] = combined_df['Monat'].apply(convert_int_to_datetime)

combined_df = extract_maschinentyp(combined_df, 'UTyp-Bezeichnung')

combined_df = replace_machine_types(combined_df, mapping_dict)

combined_df = combined_df[combined_df['Maschinentyp'].isin(allowed_types)]



# Anzeigen der ersten Zeilen des kombinierten DataFrame
print(combined_df.head())

num_rows = len(combined_df)

print("Anzahl der Zeilen im DataFrame:", num_rows)


   Claas-Planungsland Vertriebsregion Teil 2 Vertriebsregion Teil 4  Material  \
24                 ZA            6 Africa&ME         3 South AF Imp  20002040   
25                CPL            1 Middle EU                  3 CPL  20002040   
26                 RO            1 Middle EU              4 CRC Rom  20002040   
27                 FI            1 Middle EU              5 NEU Imp  20002040   
28                 AT            1 Middle EU              6 CRC Aus  20002040   

    CLAAS Untertyp SOP        UTyp-Bezeichnung      Monat  PG Fakt Ist  \
24                 110   C04/110 TUCANO 430 T3 2017-01-01            1   
25                 120  C04/120 TUCANO 420 T4f 2017-02-01            1   
26                 120  C04/120 TUCANO 420 T4f 2017-02-01            1   
27                 120  C04/120 TUCANO 420 T4f 2017-01-01            1   
28                 120  C04/120 TUCANO 420 T4f 2017-04-01            1   

   Maschinentyp  
24          L53  
25          L53  
26          L5

In [7]:
grouped_df = combined_df.groupby(['Maschinentyp', 'Monat'])

# Summiere die 'PG Fakt Ist' Spalte
mgdf = grouped_df['PG Fakt Ist'].sum().reset_index()  # Verwende reset_index, um den Index zurückzusetzen

mgdf.index = mgdf["Monat"]

mgdf['Year'] = mgdf.index.year  # Extrahieren des Jahres
mgdf['Month'] = mgdf.index.month  # Extrahieren des Monats
mgdf.drop('Monat', axis=1, inplace=True)

print(mgdf)

           Maschinentyp  PG Fakt Ist  Year  Month
Monat                                            
2016-11-01          C83            6  2016     11
2016-12-01          C83            1  2016     12
2017-01-01          C83            6  2017      1
2017-02-01          C83            5  2017      2
2017-03-01          C83            7  2017      3
...                 ...          ...   ...    ...
2022-05-01          L55           20  2022      5
2022-06-01          L55           34  2022      6
2022-07-01          L55           54  2022      7
2022-08-01          L55           36  2022      8
2022-09-01          L55           73  2022      9

[455 rows x 4 columns]


## Utils

In [5]:
# Funktion zur Konvertierung eines Datums im Integer-Format
def convert_int_to_datetime(int_date):
    date_str = f"{int_date:06d}"  # Führende Nullen hinzufügen, falls nötig
    date_str = f"{date_str[:4]}-{date_str[4:]}-01"
    return pd.to_datetime(date_str, format='%Y-%m-%d', errors='coerce')

def extract_maschinentyp(df, source_column):
    # Die Annahme ist, dass der Maschinentyp das erste Wort im String ist,
    # getrennt durch ein Leerzeichen vom Rest des Strings.
    df['Maschinentyp'] = df[source_column].str.split('/').str[0]

    return df

def replace_machine_types(df, mapping):
    df['Maschinentyp'] = df['Maschinentyp'].map(mapping).fillna(df['Maschinentyp'])
    return df

mapping = {
    'Type old': ['L02', 'L04', 'L05', 'L06', 'L07', 'L31', 'L32', 'L33', 'L34', 'L35', 
                 'L36', 'L37', 'L39', 'C02', 'C03', 'C04', 'C05', 'C07', 'C55', 'C59', 
                 'C65', 'C66', 'C73', 'C74', 'C75', 'C76'],
    'Type new': ['EVION', 'L53', 'L54', 'L55', 'L55', 'L54', 'EVION', 'EVION', 'L53', 'L54', 
                 'L55', 'L55', 'EVION', 'EVION', 'EVION', 'L53', 'L54', 'L55', 'C85', 'C86', 
                 'C85', 'C86', 'C83', 'C84', 'C85', 'C86']
}
df_mapping = pd.DataFrame(mapping)
mapping_dict = dict(zip(df_mapping['Type old'], df_mapping['Type new']))

allowed_types = ['L53', 'L54', 'L55', 'C83', 'C84', 'C85', 'C86']

